In [3]:
import sys

import numpy as np
import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.analysis.make_analysis_data as ad
import entropy.data.aggregators as agg
import entropy.data.cleaners as cl
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [4]:
month = pd.Grouper(key="date", freq="m")
idx_cols = ["user_id", month]

In [27]:
columns = [
    "User Reference",
    "Year of Birth",
    "Postcode",
    "Derived Gender",
    "Transaction Date",
    "Account Reference",
    "Provider Group Name",
    "Account Type",
    "Transaction Description",
    "Credit Debit",
    "Amount",
    "Auto Purpose Tag Name",
    "Merchant Name",
]

In [33]:
dfr = hd.read_raw_data("777", columns=None)
hd.inspect(dfr)

Time for read_raw_data                 : 3.65 seconds
(682,656, 27)


,Transaction Reference,User Reference,User Registration Date,Year of Birth,Salary Range,Postcode,LSOA,MSOA,Derived Gender,Transaction Date,Account Reference,Provider Group Name,Account Type,Latest Recorded Balance,Transaction Description,Credit Debit,Amount,User Precedence Tag Name,Manual Tag Name,Auto Purpose Tag Name,Merchant Name,Merchant Business Line,Account Created Date,Account Last Refreshed,Data Warehouse Date Created,Data Warehouse Date Last Updated,Transaction Updated Flag
0,688293,777,2011-07-20,1969.0,20K to 30K,WA1 4,E01012553,E02002603,M,2012-01-25,262916,NatWest Bank,Current,364.220001,"9572 24jan12 , tcs bowdon , bowdon gb - pos",Debit,25.030001,No Tag,No Tag,No Tag,No Merchant,Unknown Merchant,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-10-24,U
1,688291,777,2011-07-20,1969.0,20K to 30K,WA1 4,E01012553,E02002603,M,2012-01-25,262916,NatWest Bank,Current,364.220001,warrington b.c. - d/d,Debit,26.000000,Gym Membership,Gym Membership,No Tag,No Merchant,Unknown Merchant,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-10-23,U


In [34]:
dft = hd.read_txn_data("777")
hd.inspect(dft)

Time for read_txn_data                 : 3.77 seconds
(649,820, 24)


,id,date,user_id,amount,desc,merchant,tag_group,tag,user_registration_date,account_created,account_id,account_last_refreshed,account_provider,account_type,debit,desc_orig,female,latest_balance,merchant_business_line,postcode,tag_auto,tag_manual,tag_up,yob
0,688300,2012-02-01,777,400.00,mdbremoved,NaN,transfers,other_transfers,2011-07-20,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,True,<mdbremoved> - s/o,0.0,364.220001,non merchant mbl,wa1 4,transfers,other account,other account,1969.0
1,688299,2012-02-01,777,3.03,aviva pa,aviva,spend,finance,2011-07-20,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,True,aviva pa - d/d,0.0,364.220001,aviva,wa1 4,health insurance,life insurance,life insurance,1969.0


In [42]:
np.log([0, 1, 2])

/var/folders/xg/n9p73cf50s52twlnz7z778vr0000gn/T/ipykernel_2673/3719512805.py:1: RuntimeWarning: divide by zero encountered in log
  np.log([0, 1, 2])


array([      -inf, 0.        , 0.69314718])

In [5]:
dfa = hd.read_analysis_data("777")
hd.inspect(dfa)

Time for read_analysis_data            : 1.21 seconds
(729, 28)


active_accounts  txns_count  \
user_id date                                                               
13777   2012-11-30          [112648, 112650, 112652, 112651]         100   
        2012-12-31  [112648, 112650, 112652, 112651, 112653]          65   

                       txns_value  txn_count_sa  txn_count_ca  monthly_spend  \
user_id date                                                                   
13777   2012-11-30  317126.781250          14.0          85.0   11073.820312   
        2012-12-31   34068.792969           8.0          52.0    7344.680176   

                    log_monthly_spend  prop_spend_finance  \
user_id date                                                
13777   2012-11-30           9.312339            0.103978   
        2012-12-31           8.901731            0.001842   

                    prop_spend_services  prop_spend_other_spend  \
user_id date                                                      
13777   2012-11-30             0.036452                0.152251   
        2012-12-31             0.019319                0.140238   

                    prop_spend_household  prop_spend_motor  \
user_id date                                                 
13777   2012-11-30              0.647397          0.000000   
        2012-12-31              0.773029          0.027594   

                    prop_spend_communication  prop_spend_retail  \
user_id date                                                      
13777   2012-11-30                       0.0           0.045302   
        2012-12-31                       0.0           0.025045   

                    prop_spend_travel  prop_spend_hobbies  monthly_income  \
user_id date                                                                
13777   2012-11-30           0.012362            0.002258     6366.030273   
        2012-12-31           0.000000            0.012933     5283.939941   

                    annual_income  entropy_sptac      region   age  female  \
user_id date                                                                 
13777   2012-11-30   69899.824219       2.623084  South East  43.0     0.0   
        2012-12-31   69899.824219       2.705641  South East  43.0     0.0   

                    sa_inflows  sa_outflows  sa_net_inflows  \
user_id date                                                  
13777   2012-11-30     50076.0     72909.75       -22833.75   
        2012-12-31      3250.0      7837.00        -4587.00   

                    sa_scaled_inflows  sa_scaled_outflows  \
user_id date                                                
13777   2012-11-30           8.596760           12.516727   
        2012-12-31           0.557941            1.345411   

                    sa_scaled_net_inflows  
user_id date                               
13777   2012-11-30              -3.919967  
        2012-12-31              -0.787470

In [14]:
dfa[dfa.txn_count_ca == 0]

,,active_accounts,txns_count,txns_value,txn_count_sa,txn_count_ca,monthly_spend,log_monthly_spend,prop_spend_finance,prop_spend_services,prop_spend_other_spend,prop_spend_household,prop_spend_motor,prop_spend_communication,prop_spend_retail,prop_spend_travel,prop_spend_hobbies,monthly_income,annual_income,entropy_sptac,region,age,female,sa_inflows,sa_outflows,sa_net_inflows,sa_scaled_inflows,sa_scaled_outflows,sa_scaled_net_inflows
user_id,date,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [21]:
dfa.filter(regex="female|age|region").isna().sum().sum()

0

In [ ]:
pd.read_parquetjk